# Parallelization 模式演示

本notebook演示了Parallelization（并行化）模式。
这是一种通过代码协调多个LLM并行执行来提高效率的技术。

**工作流程：**
1. **Coordinator（协调器）**：代码将任务分解为多个独立组件
2. **Parallel Execution（并行执行）**：3个LLM模型同时处理不同的子任务
3. **Aggregator（聚合器）**：代码收集并汇总所有LLM的输出

**核心优势：**
- 显著提高执行效率
- 充分利用并发能力
- 适合可分解的复杂任务

## 1、加载密钥环境OpenRouter

In [1]:
import os
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("✓ API Key loaded successfully.")
else:
    print("✗ Failed to load API Key.")

✓ API Key loaded successfully.


## 2、初始化OpenRouter客户端

In [2]:
openrouter = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
print("✓ OpenRouter客户端初始化成功")

✓ OpenRouter客户端初始化成功


## 3、定义任务和子任务配置

Coordinator将大任务分解为3个可以并行处理的子任务

In [3]:
# 定义主任务
main_task = """分析一家科技创业公司在推出新产品时应该考虑的关键因素。
请从市场、技术和运营三个角度进行全面分析。"""

print("=== 主任务 ===")
print(main_task)
print("\n" + "="*70)

=== 主任务 ===
分析一家科技创业公司在推出新产品时应该考虑的关键因素。
请从市场、技术和运营三个角度进行全面分析。



## 4、Coordinator：任务分解

代码协调器将任务分解为3个独立的子任务，每个子任务分配给不同的模型

In [5]:
# Coordinator: 代码进行任务分解
subtasks = [
    {
        "id": 1,
        "name": "市场分析",
        "model": "gpt-5-mini",
        "prompt": """作为市场分析专家，请分析科技创业公司推出新产品时的市场因素：
        - 目标市场定位
        - 竞争对手分析
        - 市场需求评估
        - 定价策略
        请提供简洁但全面的分析（200-300字）。"""
    },
    {
        "id": 2,
        "name": "技术评估",
        "model": "deepseek/deepseek-chat",
        "prompt": """作为技术专家，请评估科技创业公司推出新产品时的技术因素：
        - 技术可行性
        - 开发周期
        - 技术栈选择
        - 技术风险
        请提供简洁但全面的评估（200-300字）。"""
    },
    {
        "id": 3,
        "name": "运营策略",
        "model": "anthropic/claude-sonnet-4-5",
        "prompt": """作为运营专家，请分析科技创业公司推出新产品时的运营因素：
        - 团队配置
        - 资源分配
        - 上市策略
        - 风险管理
        请提供简洁但全面的策略（200-300字）。"""
    }
]

print("\n=== Coordinator：任务分解完成 ===")
print(f"主任务已分解为 {len(subtasks)} 个子任务：\n")
for task in subtasks:
    print(f"{task['id']}. {task['name']}")
    print(f"   分配模型: {task['model']}")
    print()


=== Coordinator：任务分解完成 ===
主任务已分解为 3 个子任务：

1. 市场分析
   分配模型: gpt-5-mini

2. 技术评估
   分配模型: deepseek/deepseek-chat

3. 运营策略
   分配模型: anthropic/claude-sonnet-4-5



## 5、定义并行执行函数

创建worker函数来处理单个子任务，支持并行调用

In [6]:
def execute_subtask(subtask):
    """
    执行单个子任务
    
    参数:
        subtask: 包含任务信息的字典
    
    返回:
        包含执行结果的字典
    """
    task_id = subtask['id']
    task_name = subtask['name']
    model = subtask['model']
    prompt = subtask['prompt']
    
    print(f"⚙️  [{task_id}] 开始执行: {task_name} (模型: {model})")
    start_time = time.time()
    
    try:
        # 调用LLM
        response = openrouter.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        result = response.choices[0].message.content
        elapsed_time = time.time() - start_time
        
        print(f"✓  [{task_id}] 完成: {task_name} (耗时: {elapsed_time:.2f}秒)")
        
        return {
            "id": task_id,
            "name": task_name,
            "model": model,
            "result": result,
            "elapsed_time": elapsed_time,
            "success": True
        }
    
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"✗  [{task_id}] 失败: {task_name} - {str(e)}")
        
        return {
            "id": task_id,
            "name": task_name,
            "model": model,
            "result": f"执行失败: {str(e)}",
            "elapsed_time": elapsed_time,
            "success": False
        }

print("✓ 并行执行函数已定义")

✓ 并行执行函数已定义


## 6、并行执行所有子任务

使用ThreadPoolExecutor同时执行3个子任务，实现真正的并行化

In [7]:
print("\n" + "="*70)
print("🚀 开始并行执行所有子任务")
print("="*70)

total_start_time = time.time()
results = []

# 使用ThreadPoolExecutor进行并行执行
with ThreadPoolExecutor(max_workers=3) as executor:
    # 提交所有任务
    future_to_task = {executor.submit(execute_subtask, task): task for task in subtasks}
    
    # 收集结果（按完成顺序）
    for future in as_completed(future_to_task):
        result = future.result()
        results.append(result)

total_elapsed_time = time.time() - total_start_time

# 按任务ID排序结果
results.sort(key=lambda x: x['id'])

print("\n" + "="*70)
print(f"✓ 所有任务完成！总耗时: {total_elapsed_time:.2f}秒")
print("="*70)


🚀 开始并行执行所有子任务
⚙️  [1] 开始执行: 市场分析 (模型: gpt-5-mini)
⚙️  [2] 开始执行: 技术评估 (模型: deepseek/deepseek-chat)
⚙️  [3] 开始执行: 运营策略 (模型: anthropic/claude-sonnet-4-5)
✓  [2] 完成: 技术评估 (耗时: 7.31秒)
✓  [3] 完成: 运营策略 (耗时: 12.85秒)
✓  [1] 完成: 市场分析 (耗时: 22.25秒)

✓ 所有任务完成！总耗时: 22.25秒


## 7、展示各个子任务的结果

查看每个LLM的独立输出

In [8]:
print("\n" + "="*70)
print("📊 各子任务执行结果")
print("="*70)

for result in results:
    print(f"\n### {result['id']}. {result['name']}")
    print(f"模型: {result['model']}")
    print(f"耗时: {result['elapsed_time']:.2f}秒")
    print(f"状态: {'✓ 成功' if result['success'] else '✗ 失败'}")
    print("\n结果:")
    print("-" * 70)
    display(Markdown(result['result']))
    print("-" * 70)


📊 各子任务执行结果

### 1. 市场分析
模型: gpt-5-mini
耗时: 22.25秒
状态: ✓ 成功

结果:
----------------------------------------------------------------------


目标市场定位：按行业、规模、痛点与采纳能力细分，确定TAM/SAM/SOM与早期用户画像，选择最易验证的利基市场和有效渠道。  
竞争对手分析：识别直接/间接竞品、核心壁垒、价格与功能差距，做SWOT与替代方案映射，找出差异化角度。  
市场需求评估：通过访谈、试点、MVP与量化数据测算需求强度、增长率与客户付费意愿，考虑法规与技术趋势影响。  
定价策略：基于价值定价优先，兼用渗透或撇脂策略，设计订阅/一次性/增值（freemium）模型，测试价格弹性与渠道折扣，确保毛利与获客成本平衡。

----------------------------------------------------------------------

### 2. 技术评估
模型: deepseek/deepseek-chat
耗时: 7.31秒
状态: ✓ 成功

结果:
----------------------------------------------------------------------


在科技创业公司推出新产品时，技术因素是决定成败的关键之一。以下是对四个核心技术因素的简要评估：

1. **技术可行性**：首先需评估产品的技术可行性，确保当前技术能够支持产品功能的实现。这涉及原型验证、技术调研和可行性分析。排除技术瓶颈，确保产品在技术层面是可实现的。

2. **开发周期**：合理预估开发周期是成功的关键。开发周期过长可能导致市场机遇丧失，过短则可能影响产品质量。应综合考虑团队能力、技术复杂度和市场需求，制定切实可行的开发计划。

3. **技术栈选择**：技术栈的选择直接影响开发效率和产品性能。选择成熟、稳定且社区活跃的技术栈，有助于加快开发速度并降低维护成本。同时应考虑技术的可扩展性和团队的技术背景，确保长期可持续发展。

4. **技术风险**：识别和评估潜在技术风险至关重要。包括技术实现的难度、技术依赖的风险、安全性和性能问题等。制定风险应对策略，如技术备用方案、快速迭代和持续监控，以降低风险对项目的影响。

综合评估这些技术因素，能够帮助创业公司在新产品推出过程中，更好地平衡技术实现与市场需求的匹配，确保产品的成功落地。

----------------------------------------------------------------------

### 3. 运营策略
模型: anthropic/claude-sonnet-4-5
耗时: 12.85秒
状态: ✓ 成功

结果:
----------------------------------------------------------------------


## 科技创业公司新产品运营策略

### 团队配置
建立**精干跨职能团队**：产品经理（负责方向）、技术负责人（2-3名核心开发）、增长运营（用户获取）、客户成功（留存转化）。初期控制在5-8人，强调一人多能，避免臃肿架构。

### 资源分配
遵循**70-20-10原则**：
- 70%投入核心产品打磨（MVP验证）
- 20%用户获取渠道测试
- 10%预留应急缓冲

优先技术开发和种子用户运营，市场推广采用低成本方式（社群、内容营销）。

### 上市策略
采用**灰度发布**：
1. 内测阶段：邀请20-50名种子用户深度体验
2. 小范围公测：收集反馈快速迭代
3. 正式发布：聚焦1-2个核心渠道精准投放

强调**价值验证优先于规模扩张**。

### 风险管理
建立三层防御：
- **技术风险**：代码审查+灰度发布+实时监控
- **市场风险**：设定MVP验证指标（如留存率>40%再扩大投入）
- **财务风险**：保持6个月现金储备，单月烧钱不超预算20%

关键是**小步快跑、数据驱动**，根据用户反馈灵活调整，避免过早大规模投入。

----------------------------------------------------------------------


## 8、Aggregator：汇总所有结果

代码聚合器收集所有LLM的输出并进行汇总整合

In [9]:
print("\n" + "="*70)
print("🔄 Aggregator：开始汇总结果")
print("="*70)

# 构建综合报告
comprehensive_report = f"""
# 科技创业公司新产品推出综合分析报告

**分析任务**: {main_task}

**执行概况**:
- 并行执行的子任务数: {len(results)}
- 总执行时间: {total_elapsed_time:.2f}秒
- 成功任务数: {sum(1 for r in results if r['success'])}/{len(results)}

---

"""

# 添加每个子任务的结果
for result in results:
    comprehensive_report += f"""
## {result['id']}. {result['name']}

**执行模型**: {result['model']}  
**执行时间**: {result['elapsed_time']:.2f}秒

{result['result']}

---

"""

# 添加性能对比
comprehensive_report += """
## 性能分析

### 执行效率对比

"""

# 计算如果串行执行需要的时间
sequential_time = sum(r['elapsed_time'] for r in results)
speedup = sequential_time / total_elapsed_time if total_elapsed_time > 0 else 0

comprehensive_report += f"""
- **并行执行总时间**: {total_elapsed_time:.2f}秒
- **串行执行预估时间**: {sequential_time:.2f}秒
- **加速比**: {speedup:.2f}x
- **效率提升**: {((speedup - 1) * 100):.1f}%

### 各任务执行时间

"""

for result in results:
    bar_length = int(result['elapsed_time'] / max(r['elapsed_time'] for r in results) * 30)
    bar = '█' * bar_length
    comprehensive_report += f"- {result['name']}: {bar} {result['elapsed_time']:.2f}秒\n"

print("✓ 汇总完成")
print(f"\n综合报告长度: {len(comprehensive_report)} 字符")


🔄 Aggregator：开始汇总结果
✓ 汇总完成

综合报告长度: 1790 字符


## 9、显示综合分析报告

展示Aggregator生成的完整报告

In [10]:
display(Markdown(comprehensive_report))


# 科技创业公司新产品推出综合分析报告

**分析任务**: 分析一家科技创业公司在推出新产品时应该考虑的关键因素。
请从市场、技术和运营三个角度进行全面分析。

**执行概况**:
- 并行执行的子任务数: 3
- 总执行时间: 22.25秒
- 成功任务数: 3/3

---


## 1. 市场分析

**执行模型**: gpt-5-mini  
**执行时间**: 22.25秒

目标市场定位：按行业、规模、痛点与采纳能力细分，确定TAM/SAM/SOM与早期用户画像，选择最易验证的利基市场和有效渠道。  
竞争对手分析：识别直接/间接竞品、核心壁垒、价格与功能差距，做SWOT与替代方案映射，找出差异化角度。  
市场需求评估：通过访谈、试点、MVP与量化数据测算需求强度、增长率与客户付费意愿，考虑法规与技术趋势影响。  
定价策略：基于价值定价优先，兼用渗透或撇脂策略，设计订阅/一次性/增值（freemium）模型，测试价格弹性与渠道折扣，确保毛利与获客成本平衡。

---


## 2. 技术评估

**执行模型**: deepseek/deepseek-chat  
**执行时间**: 7.31秒

在科技创业公司推出新产品时，技术因素是决定成败的关键之一。以下是对四个核心技术因素的简要评估：

1. **技术可行性**：首先需评估产品的技术可行性，确保当前技术能够支持产品功能的实现。这涉及原型验证、技术调研和可行性分析。排除技术瓶颈，确保产品在技术层面是可实现的。

2. **开发周期**：合理预估开发周期是成功的关键。开发周期过长可能导致市场机遇丧失，过短则可能影响产品质量。应综合考虑团队能力、技术复杂度和市场需求，制定切实可行的开发计划。

3. **技术栈选择**：技术栈的选择直接影响开发效率和产品性能。选择成熟、稳定且社区活跃的技术栈，有助于加快开发速度并降低维护成本。同时应考虑技术的可扩展性和团队的技术背景，确保长期可持续发展。

4. **技术风险**：识别和评估潜在技术风险至关重要。包括技术实现的难度、技术依赖的风险、安全性和性能问题等。制定风险应对策略，如技术备用方案、快速迭代和持续监控，以降低风险对项目的影响。

综合评估这些技术因素，能够帮助创业公司在新产品推出过程中，更好地平衡技术实现与市场需求的匹配，确保产品的成功落地。

---


## 3. 运营策略

**执行模型**: anthropic/claude-sonnet-4-5  
**执行时间**: 12.85秒

## 科技创业公司新产品运营策略

### 团队配置
建立**精干跨职能团队**：产品经理（负责方向）、技术负责人（2-3名核心开发）、增长运营（用户获取）、客户成功（留存转化）。初期控制在5-8人，强调一人多能，避免臃肿架构。

### 资源分配
遵循**70-20-10原则**：
- 70%投入核心产品打磨（MVP验证）
- 20%用户获取渠道测试
- 10%预留应急缓冲

优先技术开发和种子用户运营，市场推广采用低成本方式（社群、内容营销）。

### 上市策略
采用**灰度发布**：
1. 内测阶段：邀请20-50名种子用户深度体验
2. 小范围公测：收集反馈快速迭代
3. 正式发布：聚焦1-2个核心渠道精准投放

强调**价值验证优先于规模扩张**。

### 风险管理
建立三层防御：
- **技术风险**：代码审查+灰度发布+实时监控
- **市场风险**：设定MVP验证指标（如留存率>40%再扩大投入）
- **财务风险**：保持6个月现金储备，单月烧钱不超预算20%

关键是**小步快跑、数据驱动**，根据用户反馈灵活调整，避免过早大规模投入。

---


## 性能分析

### 执行效率对比


- **并行执行总时间**: 22.25秒
- **串行执行预估时间**: 42.41秒
- **加速比**: 1.91x
- **效率提升**: 90.6%

### 各任务执行时间

- 市场分析: ██████████████████████████████ 22.25秒
- 技术评估: █████████ 7.31秒
- 运营策略: █████████████████ 12.85秒


## 10、并行化模式流程图与总结

In [ ]:
flow_diagram = f"""
```
┌────────────────────────────────────────────────────────────────┐
│                    Parallelization 模式流程图                   │
└────────────────────────────────────────────────────────────────┘

                    ┌──────────────────┐
                    │    主任务输入     │
                    └────────┬─────────┘
                             │
                             ▼
                ┌────────────────────────┐
                │  Coordinator (代码)    │  ◄── 任务分解
                │   将任务拆分为3个组件   │
                └────────────┬───────────┘
                             │
              ┌──────────────┼──────────────┐
              │              │              │
              ▼              ▼              ▼
     ┌────────────┐  ┌────────────┐  ┌────────────┐
     │ 子任务 1   │  │ 子任务 2   │  │ 子任务 3   │
     │ 市场分析   │  │ 技术评估   │  │ 运营策略   │
     └──────┬─────┘  └──────┬─────┘  └──────┬─────┘
            │                │                │
            ▼                ▼                ▼
     ┌────────────┐  ┌────────────┐  ┌────────────┐
     │ LLM-1      │  │ LLM-2      │  │ LLM-3      │
     │ GPT-Mini   │  │ DeepSeek   │  │ Claude     │
     └──────┬─────┘  └──────┬─────┘  └──────┬─────┘
            │                │                │
            │    并行执行    │    并行执行    │
            │   (同时进行)   │   (同时进行)   │
            │                │                │
            └────────────────┼────────────────┘
                             │
                             ▼
                ┌────────────────────────┐
                │  Aggregator (代码)     │  ◄── 结果汇总
                │   收集并整合所有输出    │
                └────────────┬───────────┘
                             │
                             ▼
                    ┌──────────────────┐
                    │   综合分析报告    │
                    └──────────────────┘
```

### Parallelization 模式的核心优势

1. **显著提升效率**
   - 本次执行加速比: **{speedup:.2f}x**
   - 效率提升: **{((speedup - 1) * 100):.1f}%**
   - 并行执行时间: {total_elapsed_time:.2f}秒
   - 串行执行预估: {sequential_time:.2f}秒

2. **资源优化利用**
   - 充分利用多核心和网络并发能力
   - 减少总体等待时间
   - 提高系统吞吐量

3. **任务独立性**
   - 每个子任务独立执行
   - 失败隔离，不影响其他任务
   - 易于扩展和调整

4. **灵活可控**
   - 代码完全控制协调和聚合逻辑
   - 可根据需要调整并行度
   - 支持复杂的聚合策略

### 适用场景

- **大规模数据处理**: 可分解为多个独立处理单元
- **多维度分析**: 从不同角度并行分析同一问题
- **批量内容生成**: 同时生成多个独立内容
- **实时系统**: 需要快速响应的应用

### 与串行执行对比

| 指标 | 并行执行 | 串行执行 |
|------|----------|----------|
| 总时间 | {total_elapsed_time:.2f}秒 | {sequential_time:.2f}秒 |
| 加速比 | {speedup:.2f}x | 1.0x |
| 资源利用 | 高 | 低 |
| 复杂度 | 中 | 低 |
"""

display(Markdown(flow_diagram))

## 11、保存并行化执行结果

In [ ]:
# 构建完整的执行报告
parallelization_report = {
    "main_task": main_task,
    "execution_mode": "parallel",
    "total_subtasks": len(subtasks),
    "performance": {
        "parallel_time": total_elapsed_time,
        "sequential_time_estimate": sequential_time,
        "speedup": speedup,
        "efficiency_gain_percent": ((speedup - 1) * 100)
    },
    "subtasks": [
        {
            "id": r['id'],
            "name": r['name'],
            "model": r['model'],
            "elapsed_time": r['elapsed_time'],
            "success": r['success'],
            "result_length": len(r['result'])
        }
        for r in results
    ],
    "comprehensive_report": comprehensive_report
}

# 保存到JSON文件
with open('parallelization_results.json', 'w', encoding='utf-8') as f:
    json.dump(parallelization_report, f, ensure_ascii=False, indent=2)

print("✓ 并行化执行结果已保存到 parallelization_results.json")
print(f"\n执行摘要:")
print(f"- 主任务: 科技创业公司新产品推出分析")
print(f"- 子任务数: {len(subtasks)}")
print(f"- 并行执行时间: {total_elapsed_time:.2f}秒")
print(f"- 串行执行预估: {sequential_time:.2f}秒")
print(f"- 加速比: {speedup:.2f}x")
print(f"- 效率提升: {((speedup - 1) * 100):.1f}%")

print("\n📈 各子任务执行时间:")
for r in results:
    print(f"  {r['id']}. {r['name']}: {r['elapsed_time']:.2f}秒")